# Feature Extraction

## Reading the input

In [55]:
import numpy as np
import pyedflib
from matplotlib import pyplot as plt
from nitime import utils
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from nitime.viz import plot_tseries
import csv
import pywt
import scipy.stats as sp
from spectrum import *
from os import listdir
from os.path import isfile, join

#### Names of all features

In [33]:
names = ['Min','Max','Mean','Std','Median','Activity','Mobility','Complexity','Kurtosis','2nd Difference Mean','2nd Difference Max','Coeffiecient of Variation','Skewness','1st Difference Mean','1st Difference Max',
          'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Energy',
          'Wavelet Approximate Entropy','Wavelet Detailed Entropy','Mean','Variance of Vertex to Vertex Slope','Mean','Variance of Vertex to Vertex Amplitude', 'Mean','Variance of Vertex to Vertex Time','FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower',
          'Shanon Entropy','Autro Regressive Mode Order 3 Coefficients for each channel ->']

# Mean, Median, Std, Min and Max #New

In [34]:
def mean_val(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        output[k] = np.mean(i) #saving the means
        k=k+1
    return np.sum(output)/k

def std_val(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        output[k] = np.std(i) #saving the means
        k=k+1
    return np.sum(output)/k

def max_val(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        if np.max(i)>0:
            output[k] = np.max(i)#saving the means
        else:
            output[k] = 0#saving the means
        k=k+1
    return np.sum(output)/k

def min_val(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        if np.min(i)<0:
            output[k] = np.min(i)#saving the means
        else:
            output[k] = 0#saving the means
        k=k+1
    return np.sum(output)/k

def median_val(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        output[k] = np.median(i)#saving the medians
        k=k+1
    return np.sum(output)/k

def wrapper4(a):
    mean_a =  mean_val(a)
    std_a = std_val(a)
    max_a  = max_val(a)
    min_a = min_val(a)
    median_a =  median_val(a)
    return min_a,max_a,mean_a,std_a,median_a

# Hjorth Parameters

In [35]:
def hjorth(input):                                             # function for hjorth 
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    k = 0
    for l in realinput:
        j = l[~np.isnan(l)]
        diff_input = np.diff(j)
        diff_diffinput = np.diff(diff_input)
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input)/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput)/np.var(diff_input))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.sum(hjorth_activity)/k, np.sum(hjorth_mobility)/k, np.sum(hjorth_complexity)/k                      #returning hjorth activity, hjorth mobility , hjorth complexity

# Kurtosis , 2nd Diff Mean, 2nd Diff Max

In [36]:
def my_kurtosis(a):
    b = a # Extracting the data from the 60 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 60)
    k = 0; # For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.sum(output)/k

##----------------------------------------- End Kurtosis Function ----------------------------##


##------------------------------------- Begin 2ndDiffMean(Absolute difference) Function ------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] --------------- ##
##-------------------  -- [ Output: 1D array (2ndDiffMean values for each channel)] ----------##

def secDiffMean(a):
    b = a # Extracting the data of the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for l in b:
    	t = 0.0
        i = l[~np.isnan(l)]
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.sum(output)/k

##------------------------------------- End 2ndDiffMean Function----- -------------------------##


##------------------------------------- Begin 2ndDiffMax Function(Absolute difference) --------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] -----------------##
##--------------------- [ Output: 1D array (2ndDiffMax values for each channel)] --------------##

def secDiffMax(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for l in b:
        i = l[~np.isnan(l)]
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
            	t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.sum(output)/k



def wrapper1(a):
    kurtosis =  my_kurtosis(a)
    sec_diff_mean = secDiffMean(a)
    sec_diff_max  = secDiffMax(a)
    return kurtosis,sec_diff_mean,sec_diff_max

# Coefficient of Variaition

In [37]:
def coeff_var(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for l in b:
        i = l[~np.isnan(l)]
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.sum(output)/k

# Skewness , 1st Difference Mean, 1st Difference Max

In [38]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for l in data:
        i = l[~np.isnan(l)]
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.sum(skew_array)/index


def first_diff_mean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for l in data:
        i = l[~np.isnan(l)]
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.sum(diff_mean_array)/index


def first_diff_max(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0; #current cell position in the output array
   
    for l in data:
        i = l[~np.isnan(l)]
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.sum(diff_max_array)/index


def wrapper2(arr):
    skew   = skewness(arr)
    fdmean = first_diff_mean(arr)
    fdmax  = first_diff_max(arr)
    return skew,fdmean,fdmax

# Wavelet transform features

In [39]:
def wavelet_features(epoch):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    for i in range(60):
        l=epoch[i,:]
        j = l[~np.isnan(l)]
        cA,cD=pywt.dwt(j,'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(60):   
        cA_mean.append(np.mean(cA_values[x]))
        cA_std.append(np.std(cA_values[x]))
        cA_Energy.append(np.sum(np.square(cA_values[x])))
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        cD_std.append(np.std(cD_values[x]))
        cD_Energy.append(np.sum(np.square(cD_values[x])))
        Entropy_D.append(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x]))))
        Entropy_A.append(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))
    return np.sum(cA_mean)/60,np.sum(cA_std)/60,np.sum(cD_mean)/60,np.sum(cD_std)/60,np.sum(cA_Energy)/60,np.sum(cD_Energy)/60,np.sum(Entropy_A)/60,np.sum(Entropy_D)/60

## Variance and Mean of Vertex to Vertex Slope

In [40]:
import heapq

from scipy.signal import argrelextrema

def first_diff(i):
    b=i
    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c

#first_diff(s)

def slope_mean(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for m in b:
        i = m
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = np.mean(res) 
        k=k+1
    return np.sum(output)/k




# def first_diff(i):
#     b=i
    
    
#     out = np.zeros(len(b))
    
#     for j in range(len(i)):
#         out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
#         j=j+1
#         c=out[1:len(out)]
#     return c #returns first diff


def slope_var(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for m in b:
        i = m
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]#storing maxima value
        t_max = argrelextrema(x, np.greater)[0]#storing time for maxima
        amp_min = i[argrelextrema(x, np.less)[0]]#storing minima value
        t_min = argrelextrema(x, np.less)[0]#storing time for minima value
        t = np.concatenate((t_max,t_min),axis=0) #making a single matrix of all matrix
        t.sort() #sorting according to time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q] #calculating slope        
    
        output[k] = np.var(res) 
        k=k+1#counting k
    return np.sum(output)/k

def wrapper3(epoch):
    var1 = slope_mean(epoch)
    var2 = slope_var(epoch)
    return var1,var2


## Variance and Mean of Vertex to Vertex Ampltude and Time #New

In [41]:
from scipy.signal import argrelextrema
def v2v_amp_time(a):
    b=a
    vertex_time=[]
    vertex_amplitude=[]
    for i in b:
        vertex_time.append((argrelextrema(i, np.greater))[0])
        vertex_amplitude.append(i[argrelextrema(i, np.greater)[0]])
    vertex_time_difference=[]
    vertex_amplitude_difference=[]
    variance_amplitude=[]
    mean_amplitude=[]
    mean_time=[]
    variance_time=[]
    n = len(b)
    for i in range(n):
        vertex_time_difference.append(abs(np.diff(vertex_time[i])))
        vertex_amplitude_difference.append(abs(np.diff(vertex_amplitude[i])))
    for i in range(n):
        mean_amplitude.append(np.mean(vertex_amplitude_difference[i]))
        variance_amplitude.append(np.var(vertex_amplitude_difference[i]))
        mean_time.append(np.mean(vertex_time_difference[i]))
        variance_time.append(np.var(vertex_time_difference[i]))
        
    return np.mean(mean_amplitude) , np.mean(variance_amplitude) , np.mean(mean_time) , np.mean(variance_time)

## FFT features(Max Power) 

In [42]:
from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([60,(len(BandF)-1)]);
    
    for j in range(60):
        l=data_win[j,:]
        k = l[~np.isnan(l)]
        f,Psd = signal.welch(k, Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.sum(PMax[:,0])/60,np.sum(PMax[:,1])/60,np.sum(PMax[:,2])/60,np.sum(PMax[:,3])/60


## Shanon Entropy and Entropy Spectral [Not Used]

In [43]:
def entropy(b): # Shanon Entropy
    """ Computes entropy of 0-1 vector. """
    entropy_list = []
    for labels in b:
        labels = labels[~np.isnan(labels)]
        labels = np.array([0 if x<0 else x for x in labels])
        n_labels = len(labels)
        counts = np.bincount(labels.astype(int))
        probs = counts[np.nonzero(counts)].astype(float) / n_labels
        n_classes = len(probs)

        if n_classes <= 1:
            entropy_list.append(0)
        entropy_list.append(- np.sum(probs * np.log(probs)) / np.log(n_classes))
    return np.mean(entropy_list)

## Autoregressive model

### Autoregression model- Yule Walker Algorithm [Not Used]

In [44]:
def autogressiveModelParameters(labels):
    b_labels = len(labels)
    feature = []
    for i in labels:
        k = i[~np.isnan(l)]
        coeff, sig = alg.AR_est_YW(k, 11,)
        feature.append(coeff)
    a = []     
    for i in range(11):
        a.append(np.sum(feature[:][i])/60)
     
    return a

### Autoregression model- Burg Algorithm

In [45]:
def autogressiveModelParametersBurg(labels):
    feature = []
    feature1 = []
    model_order = 3
    for i in range(60):
        l = labels[i]
        k = l[~np.isnan(l)]
        AR, rho, ref = arburg(k, model_order)
        feature.append(AR);
    for j in range(60):
        for i in range(model_order):
            feature1.append(feature[j][i])

    return feature1

## Creating the feature vectors

In [ ]:
data = csv.reader(open('l1b_mat/l1b.csv')) #data
labels = csv.reader(open('l1b_mat/labels.csv')) #labels of the data
begining = csv.reader(open('l1b_mat/begining.csv')) #Starting points for each trial

csvfile = "Features_New/l1b_features.csv"

read_file = open(csvfile, "a")

with read_file as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(names) 
    y = np.empty((0,1),str)
    X_start = np.empty((0,1),int)
    for l in labels:
        y = np.append(y, np.array([l]), axis=0)
    for l in begining:
        X_start = np.append(X_start, np.array([l]), axis=0)
    X_start = X_start.astype(float)
    X_start = X_start.astype(int)
    X_start = np.transpose(X_start)
    t=0
    count=0

    for i in X_start[0]:
        print('Reached loop no %d out of %d (%f Percent)'%((t+1),len(y),(float(t+1)*100/len(y))))
        X = np.empty((0,60), float)
        while(count<i-1):
            data.next()
            count = count+1
        while(count<i-1+2560):
            X = np.append(X, np.array([data.next()]), axis=0)
            count = count+1
        features = []
        X = np.transpose(X)
        X = X.astype(float)
        epoch = X # 256Hz signal, epoch contains signal for 10 seconds
        if len(epoch[0]) == 0:
            break

        # Min, Max, Mean, Standard Deviation, Median
        feature_list = wrapper4(epoch)
        for feat in feature_list:
            features.append(feat)
        
            
        # Hjorth Parameters
        feature_list = hjorth(epoch)
        for feat in feature_list:
            features.append(feat)

        #Kurtosis , 2nd Diff Mean, 2nd Diff Max
        feature_list = wrapper1(epoch)
        for feat in feature_list:
            features.append(feat)

        #Coeffeicient of Variation
        feat = coeff_var(epoch)
        features.append(feat)

        #Skewness , 1st Difference Mean, 1st Difference Max
        feature_list = wrapper2(epoch)
        for feat in feature_list:
            features.append(feat)


        # wavelet transform features 
        feature_list = wavelet_features(epoch)
        for feat in feature_list:
            features.append(feat)


        # Variance and mean of Vertex to Vertex Slope
        feature_list = wrapper3(epoch)
        for feat in feature_list:
            features.append(feat)
            
        # Variance and mean of Vertex to Vertex Amplitude and Time
        feature_list = v2v_amp_time(epoch)
        for feat in feature_list:
            features.append(feat)


        #Fast Fourier Transform features(Max Power) ***
        feature_list  =  maxPwelch(epoch,256)
        for feat in feature_list:
            features.append(feat)
            
        # Shanon Entropy
        features.append(entropy(epoch))

        #Autoregressive model Coefficients ***
        feature_list = autogressiveModelParametersBurg(epoch)
        for feat in feature_list:
            features.append(feat.real)

        features.append(y[t][0]);
        t = t+1

        writer.writerow(features)
read_file.close()

Reached loop no 1 out of 240 (0.416667 Percent)
Reached loop no 2 out of 240 (0.833333 Percent)
Reached loop no 3 out of 240 (1.250000 Percent)
Reached loop no 4 out of 240 (1.666667 Percent)
Reached loop no 5 out of 240 (2.083333 Percent)
Reached loop no 6 out of 240 (2.500000 Percent)
Reached loop no 7 out of 240 (2.916667 Percent)
Reached loop no 8 out of 240 (3.333333 Percent)
Reached loop no 9 out of 240 (3.750000 Percent)
Reached loop no 10 out of 240 (4.166667 Percent)
Reached loop no 11 out of 240 (4.583333 Percent)
Reached loop no 12 out of 240 (5.000000 Percent)
Reached loop no 13 out of 240 (5.416667 Percent)
Reached loop no 14 out of 240 (5.833333 Percent)
Reached loop no 15 out of 240 (6.250000 Percent)
Reached loop no 16 out of 240 (6.666667 Percent)
Reached loop no 17 out of 240 (7.083333 Percent)
Reached loop no 18 out of 240 (7.500000 Percent)
Reached loop no 19 out of 240 (7.916667 Percent)
Reached loop no 20 out of 240 (8.333333 Percent)
Reached loop no 21 out of 240

In [29]:
features[0]

0.024897484505827183

## Normalization( Wait till the csv file gets populated then run the code below after the features are extracted)

In [56]:
r = csv.reader(open('Features_New/l1b_features.csv')) # Here your csv file
lines = [l for l in r]

In [57]:
for i in range(len(lines[1])-1):  
    columns = []
    for j in range(1,len(lines)):
        columns.append(float(lines[j][i]))
    mean = np.mean(columns,axis = 0)
    std_dev  = np.std(columns,axis = 0)
    
    for j in range(1,len(lines)):
        lines[j][i] = (float(lines[j][i])-mean)/std_dev
fo = open('Features_New/l1b_Normalizedfeatures.csv', 'wb')
writer = csv.writer(fo) # This file will store the normalized features
writer.writerows(lines)
fo.close()